### Question Answer bot With Multiple Data Sources With Langchain

In [30]:
import warnings
warnings.filterwarnings('ignore')

In [31]:
# Create the Wikipedia query tool
from langchain_community.tools import WikipediaQueryRun
# Initialize Wikipedia API with result limit and content model
from langchain_community.utilities import WikipediaAPIWrapper

In [32]:
# Initialize Wikipedia API with result limit and content model
api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_model="200")
# Create the Wikipedia query tool
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name


'wikipedia'

In [33]:
# Load website content using LangChain's WebBaseLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
# Split the loaded content into smaller chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load website content using LangChain's WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/")

# Actually fetch the web content
docs = loader.load()
# Split the loaded content into smaller chunks
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
# Create a FAISS vector store from the document embeddings
vectordb = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
# Convert vector DB into retriever for question answering
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000241E40B4D60>, search_kwargs={})

In [34]:
# Wrap retriever as a LangChain tool
from langchain.tools.retriever import create_retriever_tool
# Wrap retriever as a LangChain tool
langsmith_retriver_tool = create_retriever_tool(retriever, "langsmith_search", 
                    "Search for information about LangSmith. For any questions about LangSmith, use this tool.")
langsmith_retriver_tool.name

'langsmith_search'

In [35]:
### Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_model="200")
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name


'arxiv'

In [36]:
## combine all the tools we built above
tools = [wiki, arxiv, langsmith_retriver_tool]


In [37]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()

llm = ChatGroq(model="llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000241E3E96D50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000241E3E97750>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [38]:
from langchain import hub
# Get the prompt from the hub - can be modified to add more instructions
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [39]:
#### Agents
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)


In [40]:
### Agent Executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [41]:
agent_executor.invoke({"input": "Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prot

{'input': 'Tell me about Langsmith',
 'output': "It seems like LangSmith is a platform for building production-grade Large Language Model (LLM) applications. It provides observability, evaluation, and prompt engineering tools to help developers closely monitor and evaluate their applications, and ship quickly and with confidence.\n\nSome of the key features of LangSmith include:\n\n* Observability: LangSmith allows you to analyze traces, configure metrics, dashboards, and alerts to closely monitor your application.\n* Evaluation: LangSmith provides tools to evaluate your application over production traffic, score application performance, and get human feedback on your data.\n* Prompt Engineering: LangSmith provides automatic version control and collaboration features for iterating on prompts.\n\nIt seems like LangSmith is a comprehensive platform for building and deploying LLM applications, and it's designed to help developers build high-quality applications quickly and efficiently."}

In [42]:
agent_executor.invoke({"input": "Tell me about machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.
ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.
Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusi

{'input': 'Tell me about machine learning',
 'output': 'Based on the output from the tool call, it seems that machine learning is a field of study in artificial intelligence that focuses on developing and studying statistical algorithms that can learn from data and generalize to unseen data. It finds application in many fields, including natural language processing, computer vision, speech recognition, and more.\n\nOne of the key concepts in machine learning is attention, which is a method that determines the importance of each component in a sequence relative to the other components. This is particularly useful in natural language processing, where it allows for the encoding of vectors called token embeddings across a fixed-width sequence.\n\nAnother important concept is neural networks, which are computational models inspired by the structure and functions of biological neural networks. Neural networks consist of connected units or nodes called artificial neurons, which model the neu

In [43]:
agent_executor.invoke({"input": "What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
allowed moves of arbitrary length. We show that the diameter of these graphs on
fibers of a fixed integer matrix can be bounded from above by a constant. We
then study the mixing behaviour of heat-bath random walks on these graphs. We
also state explicit conditions on the set of moves so that the heat-bath random
walk, a generalization of the Glauber dynamics, is an expander in fixed
dimension.The paper 1605.08386, titled "Heat-bath random walks with Markov bases", is about the study of graphs on lattice points with edges derived from a finite set of allowed moves of arbitrary length. The authors, Caprice Stanley and Tobias Windisch, explore the diameter and mixing behavior of heat-bath random walks on 

{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper 1605.08386, titled "Heat-bath random walks with Markov bases", is about the study of graphs on lattice points with edges derived from a finite set of allowed moves of arbitrary length. The authors, Caprice Stanley and Tobias Windisch, explore the diameter and mixing behavior of heat-bath random walks on these graphs, and provide conditions for the heat-bath random walk to be an expander in fixed dimension.'}